Dependencies

In [4]:
import os

import numpy as np

import shapely as shp
import rasterio as rio

In [5]:
work_dir = "./data/"
image = os.path.join(work_dir, "LWIR_QuickMosaic_16-bit_9327.tiff")

In [9]:
with rio.open(image) as src:
    band1 = src.read(1)
    profile = src.profile

    class32000 = np.where(band1 < 32000, 0, 1)
    class33000 = np.where(band1 < 33000, 0, 1)
    class34000 = np.where(band1 < 34000, 0, 1)
    class35000 = np.where(band1 < 35000, 0, 1)
    class36000 = np.where(band1 < 36000, 0, 1)

# raster_meta.update(dtype=rio.uint8, nodata=None)

with rio.open(os.path.join(work_dir, 'class32000.tif'), 'w', **profile) as dst: #, **raster_meta) as dst:
    dst.write(class34000.astype(rio.uint8), 1)
with rio.open(os.path.join(work_dir, 'class33000.tif'), 'w', **profile) as dst: #, **raster_meta) as dst:
    dst.write(class34000.astype(rio.uint8), 1)
with rio.open(os.path.join(work_dir, 'class34000.tif'), 'w', **profile) as dst: #, **raster_meta) as dst:
    dst.write(class34000.astype(rio.uint8), 1)
with rio.open(os.path.join(work_dir, 'class35000.tif'), 'w', **profile) as dst: #, **raster_meta) as dst:
    dst.write(class35000.astype(rio.uint8), 1)
with rio.open(os.path.join(work_dir, 'class36000.tif'), 'w', **profile) as dst: #, **raster_meta) as dst:
    dst.write(class36000.astype(rio.uint8), 1)